In [7]:
%pip install pyspark ipython-autotime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 19.4 MB/s eta 0:00:00


In [8]:
import csv
import json
import os
from pprint import pprint
import sys
import time

import numpy as np
import pandas as pd

from datetime import datetime

from pyspark import SparkConf, SparkContext

from xgboost import XGBRegressor

%load_ext autotime

time: 632 µs (started: 2024-12-12 03:10:18 +00:00)


In [9]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
time: 25.4 s (started: 2024-12-12 03:10:21 +00:00)


In [10]:
os.chdir("/content/drive/MyDrive/Colab Notebooks/DSCI553/competition")
os.getcwd()


'/content/drive/MyDrive/Colab Notebooks/DSCI553/competition'

time: 127 ms (started: 2024-12-12 03:10:48 +00:00)


In [11]:
%ls CompetitionStudentData/


business.json  photo.json         tip.json   yelp_train.csv
checkin.json   review_train.json  user.json  yelp_val.csv
time: 306 ms (started: 2024-12-12 03:10:51 +00:00)


In [12]:
%ls output/

output-final.csv
time: 211 ms (started: 2024-12-12 03:10:53 +00:00)


In [13]:
class Path:
    current_dir = os.getcwd()
    data_dir = os.path.join(current_dir, "CompetitionStudentData")
    input_csv_file = os.path.join(data_dir, "hw6_clustering.txt")

    output_dir = os.path.join(current_dir, "output")
    os.makedirs(output_dir, exist_ok=True)

    models_dir = os.path.join(current_dir, "models")
    os.makedirs(models_dir, exist_ok=True)

    task_output_file = os.path.join(output_dir, "task_op.txt")

time: 6.26 ms (started: 2024-12-12 03:10:55 +00:00)


In [14]:
def get_data_path(filename: str):
    return os.path.join(Path.data_dir, filename)

train_df = pd.read_csv(get_data_path("yelp_train.csv"))
val_df = pd.read_csv(get_data_path("yelp_val.csv"))

time: 2.29 s (started: 2024-12-12 03:10:58 +00:00)


In [15]:
print(train_df.shape)
train_df.head()

(455854, 3)


,user_id,business_id,stars
0,vxR_YV0atFxIxfOnF9uHjQ,gTw6PENNGl68ZPUpYWP50A,5.0
1,o0p-iTC5yTBV5Yab_7es4g,iAuOpYDfOTuzQ6OPpEiGwA,4.0
2,-qj9ouN0bzMXz1vfEslG-A,5j7BnXXvlS69uLVHrY9Upw,2.0
3,E43QxgV87Ij6KxMCHcijKw,jUYp798M93Mpcjys_TTgsQ,5.0
4,T13IBpJITI32a1k41rc-tg,3MntE_HWbNNoyiLGxywjYA,5.0


time: 31.5 ms (started: 2024-12-12 03:11:02 +00:00)


In [16]:
print(val_df.shape)
val_df.head(20)

(142044, 3)


,user_id,business_id,stars
0,wf1GqnKQuvH-V3QN80UOOQ,fThrN4tfupIGetkrz18JOg,5.0
1,39FT2Ui8KUXwmUt6hnwy-g,uW6UHfONAmm8QttPkbMewQ,5.0
2,7weuSPSSqYLUFga6IYP4pg,IhNASEZ3XnBHmuuVnWdIwA,4.0
3,CqaIzLiWaa-lMFYBAsYQxw,G859H6xfAmVLxbzQgipuoA,5.0
4,yy7shAsNWRbGg-8Y67Dzag,rS39YnrhoXmPqHLzCBjeqw,3.0
5,Uk1UKBIAwOqhjZdLm3r9zg,5CJL_2-XwCGBmOav4mFdYg,5.0
6,x-8ZMKKNycT3782Kqf9loA,jgtWfJCJZty_Nctqpdtp3g,5.0
7,0FVcoJko1kfZCrJRfssfIA,JVK8szNDoy9MNiYSz_MiAA,4.0
8,LcCRMIDz1JgshpPGYfLDcA,t19vb_4ML2dg5HZ-MF3muA,5.0
9,C__1BHWTGBNA5s2ZPH289g,h_UvnQfe1cuVICly_kIqHg,2.0


time: 15.3 ms (started: 2024-12-12 03:11:04 +00:00)


In [17]:
bus_df = pd.read_json(get_data_path("business.json"), lines=True, nrows=10000)
bus_df.head()

,business_id,name,neighborhood,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,Apn5Q_b6Nz61Tq4XzPdf9A,Minhas Micro Brewery,,1314 44 Avenue NE,Calgary,AB,T2E 6L6,51.091813,-114.031675,4.0,24,1,"{'BikeParking': 'False', 'BusinessAcceptsCredi...","Tours, Breweries, Pizza, Restaurants, Food, Ho...","{'Monday': '8:30-17:0', 'Tuesday': '11:0-21:0'..."
1,AjEbIBw6ZFfln7ePHha9PA,CK'S BBQ & Catering,,,Henderson,NV,89002,35.960734,-114.939821,4.5,3,0,"{'Alcohol': 'none', 'BikeParking': 'False', 'B...","Chicken Wings, Burgers, Caterers, Street Vendo...","{'Friday': '17:0-23:0', 'Saturday': '17:0-23:0..."
2,O8S5hYJ1SMc8fA4QBtVujA,La Bastringue,Rosemont-La Petite-Patrie,1335 rue Beaubien E,Montréal,QC,H2G 1K7,45.540503,-73.599300,4.0,5,0,"{'Alcohol': 'beer_and_wine', 'Ambience': '{'ro...","Breakfast & Brunch, Restaurants, French, Sandw...","{'Monday': '10:0-22:0', 'Tuesday': '10:0-22:0'..."
3,bFzdJJ3wp3PZssNEsyU23g,Geico Insurance,,211 W Monroe St,Phoenix,AZ,85003,33.449999,-112.076979,1.5,8,1,None,"Insurance, Financial Services",None
4,8USyCYqpScwiNEb58Bt6CA,Action Engine,,2005 Alyth Place SE,Calgary,AB,T2H 0N5,51.035591,-114.027366,2.0,4,1,{'BusinessAcceptsCreditCards': 'True'},"Home & Garden, Nurseries & Gardening, Shopping...","{'Monday': '8:0-17:0', 'Tuesday': '8:0-17:0', ..."


time: 906 ms (started: 2024-12-12 03:11:07 +00:00)


In [18]:

usr_df = pd.read_json(get_data_path("user.json"), lines=True, nrows=10000)
usr_df.head(10)

,user_id,name,review_count,yelping_since,friends,useful,funny,cool,fans,elite,...,compliment_more,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos
0,lzlZwIpuSWXEnNS91wxjHw,Susan,1,2015-09-28,None,0,0,0,0,None,...,0,0,0,0,0,0,0,0,0,0
1,XvLBr-9smbI0m_a7dXtB7w,Daipayan,2,2015-09-05,None,0,0,0,0,None,...,0,0,0,0,0,0,0,0,0,0
2,QPT4Ud4H5sJVr68yXhoWFw,Andy,1,2016-07-21,None,0,0,0,0,None,...,0,0,0,0,0,0,0,0,0,0
3,i5YitlHZpf0B3R0s_8NVuw,Jonathan,19,2014-08-04,None,0,0,0,0,None,...,0,0,0,0,0,0,0,0,0,0
4,s4FoIXE_LSGviTHBe8dmcg,Shashank,3,2017-06-18,None,0,0,0,0,None,...,0,0,0,0,0,0,0,0,0,0
5,ZcsZdHLiJGVvDHVjeTYYnQ,Stacey X Joe,3,2014-08-27,None,0,0,0,0,None,...,0,0,0,0,0,0,0,0,0,0
6,h3p6aeVL7vrafSOM50SsCg,Lindsay,2,2016-02-02,None,0,0,0,0,None,...,0,0,0,0,0,0,0,0,0,0
7,EbJMotYYkq-iq-v1u8wCYA,Joshua,1,2013-03-20,None,0,0,0,0,None,...,0,0,0,0,0,0,0,0,0,0
8,nnB0AE1Cxp_0154xkhXelw,Mike,11,2014-07-11,None,8,0,0,1,None,...,0,0,0,0,0,0,0,0,0,0
9,XoEnrhtJc2pcdlQ09d8Oug,Mitch,4,2013-04-07,None,2,3,1,0,None,...,0,0,0,0,0,0,0,0,0,0


time: 1.02 s (started: 2024-12-12 03:11:11 +00:00)


In [19]:
usr_df.columns


Index(['user_id', 'name', 'review_count', 'yelping_since', 'friends', 'useful',
       'funny', 'cool', 'fans', 'elite', 'average_stars', 'compliment_hot',
       'compliment_more', 'compliment_profile', 'compliment_cute',
       'compliment_list', 'compliment_note', 'compliment_plain',
       'compliment_cool', 'compliment_funny', 'compliment_writer',
       'compliment_photos'],
      dtype='object')

time: 4.69 ms (started: 2024-12-12 03:11:15 +00:00)


In [20]:
review_train_df = pd.read_json(get_data_path("review_train.json"), lines=True, nrows=50000)
review_train_df.head()

,review_id,user_id,business_id,stars,date,text,useful,funny,cool
0,D2TcNaPqXxDGQ6T0n-vHXw,YHWsLBS8jzZiPjKHMFOaAA,iKMLsX1Je7P3wAOEc9scDg,4,2011-11-07,I have been itching to get to Origin for month...,4,0,0
1,Vtd6Tqm7YMfRTBFLtZIoOQ,YHWsLBS8jzZiPjKHMFOaAA,qhJ4GDULYbdb_sctDgbZgw,2,2011-08-20,"Alright, alright, so as it turns out I sometim...",0,0,0
2,ZOBEhxOeMnncqIJFkGumRA,YHWsLBS8jzZiPjKHMFOaAA,gl1zQmiA8MUHmLL2wsCdVA,3,2011-01-10,Away from the hubbub of the Annex and Little K...,2,0,1
3,fWdhtaQld99jXB-N9yu1tg,YHWsLBS8jzZiPjKHMFOaAA,OR6iRk0vrMzE-1gLg-WYrw,2,2010-11-22,If you've ever driven by High Park on Bloor st...,9,2,4
4,ujMcxx-EzqTHht2ZtsiQ1w,YHWsLBS8jzZiPjKHMFOaAA,HOGdBz2w9VZbw2yyM-WA3g,5,2011-07-18,The High Park neighbourhood is near-void of go...,16,1,9


time: 3.23 s (started: 2024-12-12 03:11:16 +00:00)


In [21]:
review_train_df.columns


Index(['review_id', 'user_id', 'business_id', 'stars', 'date', 'text',
       'useful', 'funny', 'cool'],
      dtype='object')

time: 6.44 ms (started: 2024-12-12 03:11:23 +00:00)


In [22]:
tip_df = pd.read_json(get_data_path("tip.json"), lines=True, nrows=20000)
print(tip_df.shape)
tip_df.head()

(20000, 5)


,text,date,likes,business_id,user_id
0,Get here early enough to have dinner.,2012-07-15,0,tJRDll5yqpZwehenzE2cSg,zcTZk7OG8ovAmh_fenH21g
1,Don't use them! Double charged my boyfriend an...,2012-11-25,0,2ThtBbeDqFkVi6LugUOcVA,M5n8mMFoeXjnJEV2gxyGGA
2,Great breakfast large portions and friendly wa...,2015-08-12,0,jH19V2I9fIslnNhDzPmdkA,ZcLKXikTHYOnYt5VYRO5sg
3,Nice place. Great staff. A fixture in the tow...,2014-06-20,0,dAa0hB2yrnHzVmsCkN4YvQ,oaYhjqBbh18ZhU0bpyzSuw
4,Happy hour 5-7 Monday - Friday,2016-10-12,0,dAa0hB2yrnHzVmsCkN4YvQ,ulQ8Nyj7jCUR8M83SUMoRQ


time: 733 ms (started: 2024-12-12 03:11:25 +00:00)


In [23]:
# reviews_count = tip_df.groupby(["user_id", "business_id"]).size().reset_index(name='review_count').sort_values("review_count")
# reviews_count

reviews_count = tip_df.groupby(["user_id", "business_id"]).agg(
    review_count=("text", "size"),  # Count of reviews
    likes_sum=("likes", "sum")  # Sum of likes
).reset_index()

reviews_count.sort_values("review_count", ascending=False, inplace=True)
reviews_count.head(10)

,user_id,business_id,review_count,likes_sum
3317,BjtJ3VkMOxV2Lan037AFuw,JeHpmt1jcD2bhaZvPEQzXg,49,0
6328,PAeEkjrXTub0ENa4rZiWvA,D3dAx-QW_uuClz4MambeHA,34,0
9228,benfF2qIwxDz7TCeF1XWIA,bsXRu6wXmn_pRAGToCEPjQ,33,0
6939,QYKexxaOJQlseGWmc6soRg,siHpyAfAnoBMPpl9d_ib3A,30,0
9176,benfF2qIwxDz7TCeF1XWIA,K7j0BqmmN6ZEZ4BHIgUcng,26,0
14131,wrUYC0wc987XBBAoskOCHA,eN9LTbC9QLvkMcDtKeO9dQ,25,0
9303,blrWvPePSv87aU9hV1Zd8Q,08t3_HjbpLyPeuft6eoa5A,25,0
14494,yfK0Od09W_D5jzXBzRgnlA,JJaIocUyzeSyRSLfibHYRg,25,0
14101,wrUYC0wc987XBBAoskOCHA,YgO2v1VNtbPA7zgp0szJXA,21,0
7598,UsmTxWbobLsI6WR1Db0W7A,ESzO3Av0b1_TzKOiqzbQYQ,20,1


time: 64.9 ms (started: 2024-12-12 03:11:29 +00:00)


In [24]:
checkin_df = pd.read_json(get_data_path("checkin.json"), lines=True, nrows=10000)
checkin_df.head()

,time,business_id
0,"{'Fri-0': 2, 'Sat-0': 1, 'Sun-0': 1, 'Wed-0': ...",7KPBkxAOEtb3QeIL9PEErg
1,"{'Mon-13': 1, 'Thu-13': 1, 'Sat-16': 1, 'Wed-1...",kREVIrSBbtqBhIYkTccQUg
2,"{'Thu-0': 1, 'Mon-1': 1, 'Mon-12': 1, 'Sat-16'...",tJRDll5yqpZwehenzE2cSg
3,"{'Sun-14': 1, 'Fri-18': 1, 'Mon-20': 1}",tZccfdl6JNw-j5BKnCTIQQ
4,"{'Sat-3': 1, 'Sun-18': 1, 'Sat-21': 1, 'Sat-23...",r1p7RAMzCV_6NPF0dNoR3g


time: 581 ms (started: 2024-12-12 03:11:33 +00:00)


In [25]:
img_df = pd.read_json(get_data_path("photo.json"), lines=True, nrows=10000)
img_df.head()

,photo_id,business_id,caption,label
0,IuXwafFH3fZlTyXA-poz0w,wRKYaVXTks43GVSI2awTQA,,food
1,vhnZ58_1shy9HNVdZgtMLw,wRKYaVXTks43GVSI2awTQA,,food
2,j9ad7H2IBEzhfNCuJu4ukg,wRKYaVXTks43GVSI2awTQA,,food
3,du-5X44HccQ9Zo3pQPiFgQ,wRKYaVXTks43GVSI2awTQA,,food
4,u7Tt1nvclYNoq3UOToP-GA,wRKYaVXTks43GVSI2awTQA,The classic Farmer's Choice Breakfast has a li...,food


time: 685 ms (started: 2024-12-12 03:11:38 +00:00)


In [26]:
def save_data(data, output_file_name):
    header = ["user_id", "business_id", "prediction"]
    with open(output_file_name, "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(header)
        writer.writerows(data)


class DataReader:
    def __init__(self, sc: SparkContext, folder_path: str):
        self.sc = sc
        self.folder_path = folder_path

    def get_path(self, filename: str):
        return os.path.join(self.folder_path, filename)

    def read_csv_spark(self, path: str):
        path = self.get_path(path)
        rdd = self.sc.textFile(path)
        header = rdd.first()
        rdd = rdd.filter(lambda row: row != header).map(lambda row: row.split(","))
        return rdd, header.split()

    def read_json_spark(self, path: str):
        path = self.get_path(path)
        return self.sc.textFile(path).map(lambda row: json.loads(row))

time: 2.54 ms (started: 2024-12-12 03:11:41 +00:00)


In [27]:
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer

# Download the VADER lexicon if not already downloaded
nltk.download("vader_lexicon", quiet=True)


class BusinessData:
    keys_to_delete = [
        "name", "neighborhood", "address", "attributes", "categories", "hours", "postal_code", "city", "state"
    ]

    @staticmethod
    def parse_row(row):
        row["num_attrs"] = len(row["attributes"]) if row["attributes"] is not None else 0
        row["num_categories"] = len(row["categories"].split(",")) if row["categories"] is not None else 0
        # row["latitude"] = float(row["latitude"])
        # row["longitude"] = float(row["longitude"])
        row["stars"] = float(row["stars"])

        # Delete keys
        # row = {k: v for k, v in row.items() if k not in BusinessData.keys_to_delete}
        return row

    @staticmethod
    def generate_mapping(bus_rdd):
        # Extract unique values for state and city
        state_to_index = bus_rdd.map(lambda row: row["state"]).distinct().zipWithIndex().collectAsMap()
        city_to_index = bus_rdd.map(lambda row: row["city"]).distinct().zipWithIndex().collectAsMap()

        return state_to_index, city_to_index

    def process(rdd):
        rdd = rdd.map(lambda row: BusinessData.parse_row(row))
        rdd = rdd.map(
            lambda row: (
                row["business_id"], (
                    row["stars"], row["review_count"], row["is_open"],
                    row["num_attrs"], row["num_categories"],
                    # row["latitude"], row["longitude"]
                )
            )
        )
        # for i in rdd.take(20):
        #     print(i)
        return rdd


class UserData:
    keys_to_delete = [
        'name', "friends", 'elite', 'yelping_since',
        'compliment_hot', 'compliment_more', 'compliment_profile', 'compliment_cute',
        'compliment_list', 'compliment_note', 'compliment_plain', 'compliment_cool',
        'compliment_funny', 'compliment_writer', 'compliment_photos'
    ]
    compliment_keys = [
        'compliment_hot', 'compliment_more', 'compliment_profile', 'compliment_cute',
        'compliment_list', 'compliment_note', 'compliment_plain', 'compliment_cool',
        'compliment_funny', 'compliment_writer', 'compliment_photos'
    ]
    lc = len(compliment_keys)

    @staticmethod
    def parse_row(row):
        row["num_elite"] = len(row["elite"].split(",")) if row["elite"] != 'None' else 0
        row["num_friends"] = len(row["friends"].split(",")) if row["friends"] != 'None' else 0
        row["avg_compliment"] = sum(row[key] for key in UserData.compliment_keys) / UserData.lc

        yelping_since = datetime.strptime(row["yelping_since"], "%Y-%m-%d")
        membership_years = datetime.now() - yelping_since
        row["membership_years"] = membership_years.days / 365.25

        row["average_stars"] = float(row["average_stars"])

        # Delete keys
        # row = {k: v for k, v in row.items() if k not in UserData.keys_to_delete}
        return row

    def process(rdd):
        rdd = rdd.map(lambda row: UserData.parse_row(row)).map(
            lambda row: (
                row["user_id"], (
                    row['review_count'], row['useful'], row['funny'], row['cool'], row['fans'], row['average_stars'],
                    row['num_elite'], row['num_friends'], row['avg_compliment'], row['membership_years']
                )
            )
        )
        # for i in rdd.take(20):
        #     print(i)
        return rdd


class ReviewData:
    keys_to_delete = ['review_id', 'date', 'text']

    @staticmethod
    def parse_row(row):
        # Delete keys
        row = {k: v for k, v in row.items() if k not in ReviewData.keys_to_delete}
        return row.collectAsMap()

    def process(rdd):
        rdd = (
            rdd
            # .map(lambda row: ReviewData.parse_row(row))
            .map(
                lambda row: (
                    (row['user_id'], row['business_id']),
                    (
                        row['stars'], row['useful'], row['funny'], row['cool'], 1
                    )
                )
            )
            .reduceByKey(
                lambda x, y: (
                    x[0] + y[0],
                    x[1] + y[1],
                    x[2] + y[2],
                    x[3] + y[3],
                    x[4] + y[4],
                )  # Sum values and counts
            )
            .mapValues(
                lambda values: (
                    values[0] / values[4],
                    values[1] / values[4],
                    values[2] / values[4],
                    values[3] / values[4],
                )  # Calculate averages
            )
        )
        # for i in rdd.take(20):
        #     print(i)
        return rdd


class TipData:
    keys_to_delete = ['date', 'text']

    @staticmethod
    def parse_row(row):
        # Delete keys
        row = {k: v for k, v in row.items() if k not in TipData.keys_to_delete}
        return row

    def process(rdd):
        rdd = (
            rdd
            # .map(lambda row: TipData.parse_row(row))
            .map(
                lambda row: (
                    (row['user_id'], row['business_id']), (row['likes'], 1)
                )
            )
            .reduceByKey(
                lambda x, y: (x[0] + y[0], x[1] + y[1])
            )
        )
        # for i in rdd.take(20):
        #     print(i)
        return rdd


class PhotoData:
    keys_to_delete = ['photo_id', 'caption']
    possible_labels = ['drink', 'food', 'inside', 'menu', 'outside']

    @staticmethod
    def parse_row(row):
        # Delete keys
        row = {k: v for k, v in row.items() if k not in PhotoData.keys_to_delete}
        return row

    def process(rdd):
        rdd = (
            rdd
            # .map(lambda row: PhotoData.parse_row(row))
            .map(
                lambda row: (
                    (row['business_id']), ([row['label']], 1)
                )
            )
            .reduceByKey(
                lambda x, y: (x[0] + y[0], x[1] + y[1])
            )
            .mapValues(
                lambda values: (
                    # {label: values[0].count(label) for label in PhotoData.possible_labels},
                    len(set(values[0])),
                    values[1],
                )
            )
        )
        # for i in rdd.take(20):
        #     print(i)
        return rdd


def calculate_sentiment(text):
    # Initialize the VADER sentiment analyzer
    sia = SentimentIntensityAnalyzer()

    # Perform sentiment analysis
    sentiment_scores = sia.polarity_scores(text)

    # Return the compound sentiment score
    return sentiment_scores['compound']


def create_dataset(row, usr_dict, bus_dict, review_dict, tip_dict, img_dict):
    if len(row) == 3:
        usr, bus, rating = row
    else:
        usr, bus = row
        rating = None

    r_avg_stars, useful, funny, cool = review_dict.get((usr, bus), (3.5, 0, 0, 0))
    usr_review_count, usr_useful, usr_funny, usr_cool, usr_fans, usr_avg_stars, num_elite, num_friends, usr_avg_comp, membership_years = usr_dict.get(usr, (0, None, None, None, 0, 3.5, 0, 0, 0, None))
    bus_avg_stars, bus_review_count, bus_is_open, num_attrs, num_categories = bus_dict.get(bus, (3.5, 0, None, None, None))
    likes, upvotes = tip_dict.get((usr, bus), (0, 0))
    num_cat, num_img = img_dict.get(bus, (0, 0))

    return (
        usr, bus,
        r_avg_stars, useful, funny, cool,
        usr_review_count, usr_useful, usr_funny, usr_cool, usr_fans, usr_avg_stars, num_elite, num_friends, usr_avg_comp, membership_years,
        bus_avg_stars, bus_review_count, bus_is_open, num_attrs, num_categories,
        likes, upvotes,
        num_cat, num_img,
        float(rating)
    )

time: 1.79 s (started: 2024-12-12 03:11:44 +00:00)


In [28]:
def task(folder_path, test_file_name, output_file_name):
    start_time = time.time()

    # Initialize Spark
    conf = SparkConf().setAppName("Competition: Recommendation system")
    spark = SparkContext(conf=conf).getOrCreate()
    spark.setLogLevel("ERROR")

    try:
        data_reader = DataReader(spark, folder_path)

        # Additional Data
        usr_rdd = data_reader.read_json_spark("user.json")
        usr_rdd = UserData.process(usr_rdd)
        usr_rdd = usr_rdd.cache().collectAsMap()

        bus_rdd = data_reader.read_json_spark("business.json")
        bus_rdd = BusinessData.process(bus_rdd)
        bus_rdd = bus_rdd.cache().collectAsMap()

        review_rdd = data_reader.read_json_spark("review_train.json")
        review_rdd = ReviewData.process(review_rdd)
        review_rdd = review_rdd.cache().collectAsMap()

        tip_rdd = data_reader.read_json_spark("tip.json")
        tip_rdd = TipData.process(tip_rdd)
        tip_rdd = tip_rdd.cache().collectAsMap()

        img_rdd = data_reader.read_json_spark("photo.json")
        img_rdd = PhotoData.process(img_rdd)
        img_rdd = img_rdd.cache().collectAsMap()

        # cin_rdd = data_reader.read_json_spark("checkin.json")

        train_rdd, _ = data_reader.read_csv_spark("yelp_train.csv")

        # Read validation dataset
        test_file_name = os.path.basename(test_file_name)
        val_rdd, _ = data_reader.read_csv_spark(test_file_name)

        # Merge datasets
        train_rdd = train_rdd.map(lambda row: create_dataset(row, usr_rdd, bus_rdd, review_rdd, tip_rdd, img_rdd))
        val_processed = val_rdd.map(lambda row: create_dataset(row, usr_rdd, bus_rdd, review_rdd, tip_rdd, img_rdd))

        column_names = [
            "user_id", "business_id",
            "review_avg_stars", "useful", "funny", "cool",
            "usr_review_count", "usr_useful", "usr_funny", "usr_cool", "usr_fans", "usr_avg_stars", "num_elite", "num_friends", "usr_avg_comp", "membership_years",
            "bus_avg_stars", "bus_review_count", "bus_is_open", "num_attrs", "num_categories",
            "likes", "upvotes",
            "num_cat", "num_img",
            "rating"
        ]

        train_df_processed = pd.DataFrame(train_rdd.collect(), columns=column_names)
        train_df_processed.to_csv("yelp_train_processed.csv", index=False)

        val_df_processed = pd.DataFrame(val_processed.collect(), columns=column_names)
        val_df_processed.to_csv("yelp_val_processed.csv", index=False)
        # # Extract X_train and Y_train
        # X_train = train_rdd.map(lambda x: x[1]).cache()
        # X_train = np.array(X_train.collect(), dtype="float32")
        # Y_train = train_rdd.map(lambda x: x[2]).cache()
        # Y_train = np.array(Y_train.collect(), dtype="float32")

        # # Extract X_train and Y_train
        # X_val = val_processed.map(lambda x: x[1]).cache()
        # X_val = np.array(X_val.collect(), dtype="float32")

        # xgb = XGBRegressor()
        # xgb.fit(X_train, Y_train)
        # Y_pred = xgb.predict(X_val)

        # pred_data = []
        # for i, row in enumerate(val_rdd.collect()):
        #     pred_data.append([row[0], row[1], Y_pred[i]])

        # save_data(pred_data, output_file_name)

    except Exception as e:
        print(f"Exception occured:\n{e}")

    finally:
        spark.stop()

    execution_time = time.time() - start_time
    print(f"Duration: {execution_time}\n")


# if __name__ == "__main__":
#     if len(sys.argv) != 4:
#         print("Usage: spark-submit competition.py   ")
#         sys.exit(1)

#     # Read input parameters
#     folder_path = sys.argv[1]
#     test_file_name = sys.argv[2]
#     output_file_name = sys.argv[3]

#     task(folder_path, test_file_name, output_file_name)

task("CompetitionStudentData", "CompetitionStudentData/yelp_val.csv", "output/output-1.csv")


Exception occured:
An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.hadoop.mapred.InvalidInputException: Input path does not exist: file:/content/CompetitionStudentData/user.json
	at org.apache.hadoop.mapred.FileInputFormat.singleThreadedListStatus(FileInputFormat.java:304)
	at org.apache.hadoop.mapred.FileInputFormat.listStatus(FileInputFormat.java:244)
	at org.apache.hadoop.mapred.FileInputFormat.getSplits(FileInputFormat.java:332)
	at org.apache.spark.rdd.HadoopRDD.getPartitions(HadoopRDD.scala:208)
	at org.apache.spark.rdd.RDD.$anonfun$partitions$2(RDD.scala:294)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.rdd.RDD.partitions(RDD.scala:290)
	at org.apache.spark.rdd.MapPartitionsRDD.getPartitions(MapPartitionsRDD.scala:49)
	at org.apache.spark.rdd.RDD.$anonfun$partitions$2(RDD.scala:294)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.rdd.RDD.partitions(RDD.scala:290)
	at org.apache.spark.api

In [29]:
train_df_processed = pd.read_csv("yelp_train_processed.csv")
val_df_processed = pd.read_csv("yelp_val_processed.csv")

time: 4.73 s (started: 2024-12-12 03:12:07 +00:00)


In [ ]:
val_df_processed


,user_id,business_id,review_avg_stars,useful,funny,cool,usr_review_count,usr_useful,usr_funny,usr_cool,...,bus_avg_stars,bus_review_count,bus_is_open,num_attrs,num_categories,likes,upvotes,num_cat,num_img,rating
0,wf1GqnKQuvH-V3QN80UOOQ,fThrN4tfupIGetkrz18JOg,3.5,0,0,0,118,2,0,1,...,3.5,122,1,26,7,0,0,3,11,5.0
1,39FT2Ui8KUXwmUt6hnwy-g,uW6UHfONAmm8QttPkbMewQ,3.5,0,0,0,141,12,3,3,...,4.5,1264,1,22,4,0,0,4,46,5.0
2,7weuSPSSqYLUFga6IYP4pg,IhNASEZ3XnBHmuuVnWdIwA,3.5,0,0,0,251,46,5,9,...,5.0,1380,1,11,6,0,0,0,0,4.0
3,CqaIzLiWaa-lMFYBAsYQxw,G859H6xfAmVLxbzQgipuoA,3.5,0,0,0,104,14,15,3,...,4.5,112,1,22,11,0,0,3,14,5.0
4,yy7shAsNWRbGg-8Y67Dzag,rS39YnrhoXmPqHLzCBjeqw,3.5,0,0,0,256,129,40,8,...,3.0,171,1,25,6,0,0,1,2,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142039,pA9NXgASl86RImkdBtydrA,q6-SF8zHFU1AWO70k92o1Q,3.5,0,0,0,56,24,6,22,...,3.5,219,1,5,2,0,0,0,0,2.0
142040,_eUb7UGsUoSfi9n2ieF5ow,hgWMxKhrnOUd3m5nOUBIkA,3.5,0,0,0,59,36,3,0,...,3.0,350,1,6,4,0,0,0,0,4.0
142041,cEJGXB63KhROA-XmE_jgXw,0ldxjei8v4q95fApIei3Lg,3.5,0,0,0,123,13,1,1,...,4.0,904,1,25,6,0,0,4,28,5.0
142042,Z4-V0hc51oxUdULWJOufeg,j29tuUdrfaxmGjwxHdHZPA,3.5,0,0,0,55,4,1,0,...,4.0,104,1,20,4,0,0,2,4,3.0


time: 60.7 ms (started: 2024-12-11 21:51:11 +00:00)


In [30]:
val_df_processed.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 142044 entries, 0 to 142043
Data columns (total 26 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   user_id           142044 non-null  object 
 1   business_id       142044 non-null  object 
 2   review_avg_stars  0 non-null       float64
 3   useful            142044 non-null  int64  
 4   funny             142044 non-null  int64  
 5   cool              142044 non-null  int64  
 6   usr_review_count  142044 non-null  int64  
 7   usr_useful        142044 non-null  int64  
 8   usr_funny         142044 non-null  int64  
 9   usr_cool          142044 non-null  int64  
 10  usr_fans          142044 non-null  int64  
 11  usr_avg_stars     142044 non-null  float64
 12  num_elite         142044 non-null  int64  
 13  num_friends       142044 non-null  int64  
 14  usr_avg_comp      142044 non-null  float64
 15  membership_years  142044 non-null  float64
 16  bus_avg_stars     14

In [31]:
from sklearn.model_selection import cross_val_score, StratifiedKFold
from xgboost import XGBRegressor
from sklearn.metrics import make_scorer, mean_squared_error
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor


drop_cols = [
    "user_id", "business_id", "rating",
    "review_avg_stars", "useful", "funny", "cool",
    # "num_attrs",
    # "num_categories"
    # "likes", "upvotes",
    # "num_cat", "num_img"
]

# Define the number of folds
n_splits = 10

param = {
    'lambda': 9.92724463758443,
    'alpha': 0.2765119705933928,
    'colsample_bytree': 0.5,
    'subsample': 0.8,
    'learning_rate': 0.02,
    'max_depth': 17,
    'random_state': 2020,
    'min_child_weight': 101,
    'n_estimators': 300,
}


time: 347 ms (started: 2024-12-12 03:12:23 +00:00)


In [32]:
# Split features (X) and target (y)
X_train = train_df_processed.drop(columns=drop_cols)
y_train = train_df_processed['rating']
X_test = val_df_processed.drop(columns=drop_cols)
y_test = val_df_processed['rating']

scaler = MinMaxScaler()

# Initialize the stratified k-fold splitter
stratified_kfold = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# Initialize a list to store evaluation metrics for each fold
metrics = []
metrics_test = []

# Iterate over the folds
for fold, (train_index, val_index) in enumerate(stratified_kfold.split(X_train, y_train)):
    X_train_split, X_val_split = X_train.iloc[train_index], X_train.iloc[val_index]
    y_train_split, y_val_split = y_train.iloc[train_index], y_train.iloc[val_index]

    # Initialize and train the XGBRegressor model
    model = XGBRegressor()
    model.fit(X_train_split, y_train_split)

    # Make predictions on the validation set
    y_pred_split = model.predict(X_val_split)
    y_test_pred = model.predict(X_test)

    # Calculate RMSE
    rmse_val = np.sqrt(mean_squared_error(y_val_split, y_pred_split))
    rmse_test = np.sqrt(mean_squared_error(y_test, y_test_pred))
    print(f"Fold: {fold+1} Validation: {rmse_val}, Test: {rmse_test}")

    # Store the evaluation metric for this fold
    metrics.append(rmse_val)
    metrics_test.append(rmse_test)

# Calculate the average RMSE over all folds
print("Avg Validation RMSE:", np.mean(metrics), "Avg Test RMSE:", np.mean(metrics_test))


Fold: 1 Validation: 0.9825470150904482, Test: 0.979583677774133
Fold: 2 Validation: 0.9797771402280292, Test: 0.9795007399167855
Fold: 3 Validation: 0.9819861470305652, Test: 0.9801539497128529
Fold: 4 Validation: 0.9821612963382543, Test: 0.9799708199418334
Fold: 5 Validation: 0.9866627020185416, Test: 0.9798621853549565
Fold: 6 Validation: 0.9834289667092033, Test: 0.979787262093987
Fold: 7 Validation: 0.9802584242082636, Test: 0.9795139451726801
Fold: 8 Validation: 0.9837289881055956, Test: 0.9802602163124784
Fold: 9 Validation: 0.9830276896358469, Test: 0.9801862233848292
Fold: 10 Validation: 0.9838610929944122, Test: 0.9795864568224455
Avg Validation RMSE: 0.9827439462359159 Avg Test RMSE: 0.9798405476486982
time: 1min 26s (started: 2024-12-12 03:12:34 +00:00)


In [33]:
# Define the number of folds
n_splits = 10

# Initialize min-max scaler
scaler = MinMaxScaler()

# Apply min-max normalization to the training and test data
X_train_norm = scaler.fit_transform(train_df_processed.drop(columns=drop_cols))
y_train = train_df_processed['rating']
X_test_norm = scaler.transform(val_df_processed.drop(columns=drop_cols))
y_test = val_df_processed['rating']

# Initialize the stratified k-fold splitter
stratified_kfold = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# Initialize a list to store evaluation metrics for each fold
metrics = []
metrics_test = []

# Iterate over the folds
for fold, (train_index, val_index) in enumerate(stratified_kfold.split(X_train_norm, y_train)):
    X_train_split, X_val_split = X_train_norm[train_index], X_train_norm[val_index]
    y_train_split, y_val_split = y_train.iloc[train_index], y_train.iloc[val_index]

    # Initialize and train the XGBRegressor model
    model = XGBRegressor(**param)
    model.fit(X_train_split, y_train_split)

    # Make predictions on the validation set
    y_pred_split = model.predict(X_val_split)
    y_test_pred = model.predict(X_test_norm)

    # Calculate RMSE
    rmse_val = np.sqrt(mean_squared_error(y_val_split, y_pred_split))
    rmse_test = np.sqrt(mean_squared_error(y_test, y_test_pred))
    print(f"Fold: {fold+1} Validation: {rmse_val}, Test: {rmse_test}")

    # Store the evaluation metric for this fold
    metrics.append(rmse_val)
    metrics_test.append(rmse_test)

# Calculate the average RMSE over all folds
print("Avg Validation RMSE:", np.mean(metrics), "Avg Test RMSE:", np.mean(metrics_test))


Fold: 1 Validation: 0.9812227654430764, Test: 0.9785357481163717
Fold: 2 Validation: 0.9787980374943659, Test: 0.9783984669680085
Fold: 3 Validation: 0.9815621240414991, Test: 0.9785258190682441
Fold: 4 Validation: 0.9801809531573265, Test: 0.9787232042076939
Fold: 5 Validation: 0.9846379907082266, Test: 0.9783949709119769
Fold: 6 Validation: 0.9822843846397403, Test: 0.9785590252948744
Fold: 7 Validation: 0.9775793619950669, Test: 0.978354501642081
Fold: 8 Validation: 0.9814670245085647, Test: 0.9785495567600142
Fold: 9 Validation: 0.981112859090889, Test: 0.978764027835619
Fold: 10 Validation: 0.9829397364188949, Test: 0.9785309837361318
Avg Validation RMSE: 0.981178523749765 Avg Test RMSE: 0.9785336304541016
time: 15min 24s (started: 2024-12-12 03:14:08 +00:00)


In [34]:
# Initialize min-max scaler
scaler = MinMaxScaler()

# Apply min-max normalization to the training and test data
X_train_norm = scaler.fit_transform(train_df_processed.drop(columns=drop_cols))
y_train = train_df_processed['rating']
X_test_norm = scaler.transform(val_df_processed.drop(columns=drop_cols))
y_test = val_df_processed['rating']

model = XGBRegressor(**param)
model.fit(X_train_norm, y_train)

y_test_pred = model.predict(X_test_norm)
rmse_test = np.sqrt(mean_squared_error(y_test, y_test_pred))
print(rmse_test)

0.9780868823994201
time: 1min 32s (started: 2024-12-12 03:32:11 +00:00)


In [36]:
from sklearn.metrics import mean_squared_error

op = pd.read_csv("output/output-final.csv")

# Calculate RMSE
rmse = np.sqrt(mean_squared_error(val_df_processed["rating"], op["prediction"]))
print(rmse)

0.9780868823994201
time: 2.02 s (started: 2024-12-12 03:46:20 +00:00)


In [37]:
op["y_true"] = val_df_processed["rating"]
op["error"] = abs(op["y_true"] - op["prediction"])
bins = [-np.inf, 1, 2, 3, 4, np.inf]

# Define the labels for error levels
labels = ['>=0 and <1:', '>=1 and <2:', '>=2 and <3:', '>=3 and <4:', '>=4:']

# Bin the absolute differences into error levels
op['Error Distribution:'] = pd.cut(op['error'], bins=bins, labels=labels, right=False)

# Count the occurrences in each error level
error_distribution = op['Error Distribution:'].value_counts().sort_index()

print(error_distribution)


Error Distribution:
>=0 and <1:    102076
>=1 and <2:     33039
>=2 and <3:      6156
>=3 and <4:       772
>=4:                1
Name: count, dtype: int64
time: 23.4 ms (started: 2024-12-12 03:46:26 +00:00)


In [44]:
# Method Description

# Model-Based Recommendation System for Yelp Reviews Data**

# This script implements a model-based recommendation system for Yelp reviews data. It aims to predict user ratings for
# businesses based on various features extracted from the Yelp Reviews dataset.

# ## Model
# To enhance the effectiveness of our recommendation system, I opted for a model-based approach over the hybrid method
# utilized in our previous work (HW3). This decision stems from the superior performance of model-based systems across
# various components of the recommendation process. I employ the XGBoost Regressor in this implementation, utilizing
# parameters optimized through local GridSearch.

# ## Feature Engineering:
# Our feature selection process draws from diverse aspects of the Yelp dataset, encompassing user-centric metrics
# (e.g., review count, average stars, elite status), business-oriented attributes (e.g., review count, average stars,
# attributes), and the dynamics of user-business interactions (e.g., reviews, tips). Additionally, I consider
# metadata from business-associated photos as an indicator of authenticity and trustworthiness. Throughout feature
# extraction, we employ aggregation techniques to consolidate sparse matrices and mitigate null entries.

# After extensive experimentation with different feature sets, we made strategic decisions regarding feature inclusion.
# While initially excluding review-related features due to their absence for (user, business) pairs in validation and
# test sets, reintroducing them became imperative after observing adverse effects on RMSE upon removal. Similarly,
# their reinstatement was necessary to maintain model performance despite the initial exclusion of tip and photo
# features.
#

#######################################################################################################################
# Error Distribution
#######################################################################################################################
#
# >=0 and <1:    102162
# >=1 and <2:     32993
# >=2 and <3:      6116
# >=3 and <4:       773
# >=4:                0
#

#######################################################################################################################
# RMSE
#######################################################################################################################
#
# Validation Set:         0.9772555489644983, good
# Test Set:
#

#######################################################################################################################
# Execution Time
#######################################################################################################################
#
# Data Processing Time:       183.61311721801758 s
# Model Training Time:        425.02053117752075 s
# -----------------------------------------------------------
# Total Execution Time:       622.1228864192963 s
#


import os
import sys
import csv
import time
import json
import numpy as np
import pandas as pd
from datetime import datetime
from xgboost import XGBRegressor
from pyspark import SparkConf, SparkContext
from sklearn.preprocessing import MinMaxScaler


class Path:
    yelp_train_processed = "yelp_train_processed.csv"
    yelp_val_processed = "yelp_val_processed.csv"


class DataReader:
    def __init__(self, sc: SparkContext):
        self.sc = sc

    def read_csv_spark(self, path: str):
        rdd = self.sc.textFile(path)
        header = rdd.first()
        data_rdd = rdd.filter(lambda row: row != header)
        split_rdd = data_rdd.map(lambda row: row.split(","))
        header_columns = header.split()
        return split_rdd, header_columns


    def read_json_spark(self, path: str):
        rdd = self.sc.textFile(path)
        json_rdd = rdd.map(lambda row: json.loads(row))
        return json_rdd


class BusinessData:
    keys_to_delete = ["name", "neighborhood", "address", "attributes", "categories", "hours", "postal_code", "city", "state"]

    @staticmethod
    def parse_row(row: dict):
        row["num_attrs"] = len(row["attributes"]) if row["attributes"] is not None else 0
        row["num_categories"] = len(row["categories"].split(",")) if row["categories"] is not None else 0
        row["stars"] = float(row["stars"])
        return row

    @staticmethod
    def generate_mapping(rdd):
        state_to_index = rdd.map(lambda row: row["state"]).distinct().zipWithIndex().collectAsMap()
        city_to_index = rdd.map(lambda row: row["city"]).distinct().zipWithIndex().collectAsMap()
        return state_to_index, city_to_index

    def process(rdd):
        rdd = rdd.map(lambda row: BusinessData.parse_row(row))
        rdd = rdd.map(lambda row: (row["business_id"], (row["stars"], row["review_count"], row["is_open"], row["num_attrs"], row["num_categories"])))
        return rdd.cache().collectAsMap()


class UserData:
    keys_to_delete = ["name", "friends", "elite", "yelping_since", "compliment_hot", "compliment_more", "compliment_profile", "compliment_cute", "compliment_list", "compliment_note", "compliment_plain", "compliment_cool", "compliment_funny", "compliment_writer", "compliment_photos"]
    compliment_keys = ["compliment_hot", "compliment_more", "compliment_profile", "compliment_cute", "compliment_list", "compliment_note", "compliment_plain", "compliment_cool", "compliment_funny", "compliment_writer", "compliment_photos"]
    lc = len(compliment_keys)

    @staticmethod
    def parse_row(row: dict):
        row["num_elite"] = len(row["elite"].split(",")) if row["elite"] != "None" else 0
        row["num_friends"] = len(row["friends"].split(",")) if row["friends"] != "None" else 0
        row["avg_compliment"] = sum(row[key] for key in UserData.compliment_keys) / UserData.lc

        yelping_since = datetime.strptime(row["yelping_since"], "%Y-%m-%d")
        membership_years = datetime.now() - yelping_since
        row["membership_years"] = membership_years.days / 365.25

        row["average_stars"] = float(row["average_stars"])
        return row

    def process(rdd):
        return rdd.map(lambda row: UserData.parse_row(row)).map(
            lambda row: (row["user_id"], (row["review_count"], row["useful"], row["funny"], row["cool"], row["fans"], row["average_stars"], row["num_elite"], row["num_friends"], row["avg_compliment"], row["membership_years"]))
        ).cache().collectAsMap()


class ReviewData:
    keys_to_delete = ["review_id", "date", "text"]

    @staticmethod
    def parse_row(row): return {k: v for k, v in row.items() if k not in ReviewData.keys_to_delete}

    def process(rdd):
        return rdd.map(lambda row: ((row["user_id"], row["business_id"]), (row["stars"], row["useful"], row["funny"], row["cool"], 1))) \
                  .reduceByKey(lambda x, y: tuple(xi + yi for xi, yi in zip(x, y))) \
                  .mapValues(lambda values: tuple(v / values[4] for v in values[:4])) \
                  .cache().collectAsMap()


class TipData:
    keys_to_delete = ["date", "text"]

    @staticmethod
    def parse_row(row):
        row = {k: v for k, v in row.items() if k not in TipData.keys_to_delete}
        return row

    def process(rdd):
        rdd = (
            rdd
            .map(lambda row: ((row["user_id"], row["business_id"]), (row["likes"], 1))).reduceByKey(
                lambda x, y: (x[0] + y[0], x[1] + y[1])
            )
        )
        return rdd.cache().collectAsMap()


class PhotoData:
    keys_to_delete = ["photo_id", "caption"]
    possible_labels = ["drink", "food", "inside", "menu", "outside"]

    @staticmethod
    def parse_row(row):
        row = {k: v for k, v in row.items() if k not in PhotoData.keys_to_delete}
        return row

    def process(rdd):
        rdd = (
            rdd
            .map(lambda row: ((row["business_id"]), ([row["label"]], 1)))
            .reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1]))
            .mapValues(
                lambda values: (
                    len(set(values[0])),
                    values[1],
                )
            )
        )
        return rdd.cache().collectAsMap()


class ModelBasedConfig:
    drop_cols = ["user_id", "business_id", "rating", "review_avg_stars", "useful", "funny", "cool"]
    params = {"lambda": 9.92724463758443, "alpha": 0.2765119705933928, "colsample_bytree": 0.5, "subsample": 0.8,
              "learning_rate": 0.02, "max_depth": 17, "random_state": 2020, "min_child_weight": 101, "n_estimators": 300}
    pred_cols = ["user_id", "business_id", "prediction"]


def create_dataset(row, usr_dict, bus_dict, review_dict, tip_dict, img_dict):
    usr, bus, rating = (row + [None])[:3]
    r_avg_stars, useful, funny, cool = review_dict.get((usr, bus), (None, 0, 0, 0))
    usr_data = usr_dict.get(usr, (0, None, None, None, 0, 3.5, 0, 0, 0, None))
    bus_data = bus_dict.get(bus, (3.5, 0, None, None, None))
    likes, upvotes = tip_dict.get((usr, bus), (0, 0))
    num_cat, num_img = img_dict.get(bus, (0, 0))

    return (usr, bus, r_avg_stars, useful, funny, cool, *usr_data, *bus_data, likes, upvotes, num_cat, num_img, float(rating))


def save_data(data: list, output_file_name: str):
    header = ["user_id", "business_id", "prediction"]
    with open(output_file_name, "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(header)
        writer.writerows(data)


def process_data(folder_path: str, test_file_name: str):
    start_time = time.time()
    conf = SparkConf().setAppName("Competition: Recommendation system")
    spark = SparkContext(conf=conf).getOrCreate()
    spark.setLogLevel("ERROR")

    try:
        data_reader = DataReader(spark)

        usr_rdd = UserData.process(data_reader.read_json_spark(os.path.join(folder_path, "user.json")))
        bus_rdd = BusinessData.process(data_reader.read_json_spark(os.path.join(folder_path, "business.json")))
        review_rdd = ReviewData.process(data_reader.read_json_spark(os.path.join(folder_path, "review_train.json")))
        tip_rdd = TipData.process(data_reader.read_json_spark(os.path.join(folder_path, "tip.json")))
        img_rdd = PhotoData.process(data_reader.read_json_spark(os.path.join(folder_path, "photo.json")))

        train_rdd, _ = data_reader.read_csv_spark(os.path.join(folder_path, "yelp_train.csv"))
        val_rdd, _ = data_reader.read_csv_spark(test_file_name)

        train_processed = train_rdd.map(lambda row: create_dataset(row, usr_rdd, bus_rdd, review_rdd, tip_rdd, img_rdd))
        val_processed = val_rdd.map(lambda row: create_dataset(row, usr_rdd, bus_rdd, review_rdd, tip_rdd, img_rdd))

        column_names = [
            "user_id", "business_id", "review_avg_stars", "useful", "funny", "cool", "usr_review_count",
            "usr_useful", "usr_funny", "usr_cool", "usr_fans", "usr_avg_stars", "num_elite", "num_friends",
            "usr_avg_comp", "membership_years", "bus_avg_stars", "bus_review_count", "bus_is_open",
            "num_attrs", "num_categories", "likes", "upvotes", "num_cat", "num_img", "rating"
        ]

        pd.DataFrame(train_processed.collect(), columns=column_names).to_csv(Path.yelp_train_processed, index=False)
        pd.DataFrame(val_processed.collect(), columns=column_names).to_csv(Path.yelp_val_processed, index=False)

    except Exception as e:
        print(f"Exception occurred: {e}")
    finally:
        spark.stop()

    print(f"Data Processing Duration: {time.time() - start_time} s\n")


def train_model(train_data_path: str, test_data_path: str):
    start_time = time.time()
    train_df_processed = pd.read_csv(train_data_path)
    val_df_processed = pd.read_csv(test_data_path)

    scaler = MinMaxScaler()

    X_train_norm = scaler.fit_transform(train_df_processed.drop(columns=ModelBasedConfig.drop_cols))
    y_train = train_df_processed["rating"]
    X_test_norm = scaler.transform(val_df_processed.drop(columns=ModelBasedConfig.drop_cols))

    model = XGBRegressor(**ModelBasedConfig.params)
    model.fit(X_train_norm, y_train)

    y_test_pred = model.predict(X_test_norm)
    val_df_processed["prediction"] = y_test_pred

    pred_df = val_df_processed.loc[:, ModelBasedConfig.pred_cols]

    execution_time = time.time() - start_time
    print(f"Model Training Time: {execution_time} s\n")

    return pred_df.values.tolist()


def get_error_distribution(test_data_path: str, output_file_name: str):
    val_df_processed = pd.read_csv(test_data_path, usecols=["user_id", "business_id", "rating"])

    pred_df = pd.read_csv(output_file_name)
    pred_df["error"] = abs(val_df_processed["rating"] - pred_df["prediction"])

    bins = [-np.inf, 1, 2, 3, 4, np.inf]
    labels = [">=0 and <1:", ">=1 and <2:", ">=2 and <3:", ">=3 and <4:", ">=4:"]

    pred_df["Error Distribution:"] = pd.cut(pred_df["error"], bins=bins, labels=labels, right=False)

    error_distribution = pred_df["Error Distribution:"].value_counts().sort_index()
    print(error_distribution)


def main(folder_path: str, test_file_name: str, output_file_name: str):
    start_time = time.time()
    process_data(folder_path, test_file_name)

    pred_data = train_model(Path.yelp_train_processed, Path.yelp_val_processed)

    save_data(pred_data, output_file_name)

    get_error_distribution(Path.yelp_val_processed, output_file_name)

    execution_time = time.time() - start_time
    print(f"Total Execution Time: {execution_time} s\n")

if __name__ == "__main__":
    if len(sys.argv) != 4:
        print("Usage: spark-submit competition.py <folder_path> <test_file_name> <output_file_name>")
        sys.exit(1)

    folder_path = sys.argv[1]
    test_file_name = sys.argv[2]
    output_file_name = sys.argv[3]

    #main(folder_path, test_file_name, output_file_name)


Usage: spark-submit competition.py <folder_path> <test_file_name> <output_file_name>


SystemExit: 1

time: 16.1 ms (started: 2024-12-12 03:59:28 +00:00)


/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [45]:
main("CompetitionStudentData", "CompetitionStudentData/yelp_val.csv", "output/output-final.csv")


Exception occurred: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.hadoop.mapred.InvalidInputException: Input path does not exist: file:/content/CompetitionStudentData/user.json
	at org.apache.hadoop.mapred.FileInputFormat.singleThreadedListStatus(FileInputFormat.java:304)
	at org.apache.hadoop.mapred.FileInputFormat.listStatus(FileInputFormat.java:244)
	at org.apache.hadoop.mapred.FileInputFormat.getSplits(FileInputFormat.java:332)
	at org.apache.spark.rdd.HadoopRDD.getPartitions(HadoopRDD.scala:208)
	at org.apache.spark.rdd.RDD.$anonfun$partitions$2(RDD.scala:294)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.rdd.RDD.partitions(RDD.scala:290)
	at org.apache.spark.rdd.MapPartitionsRDD.getPartitions(MapPartitionsRDD.scala:49)
	at org.apache.spark.rdd.RDD.$anonfun$partitions$2(RDD.scala:294)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.rdd.RDD.partitions(RDD.scala:290)
	at org.apache.spark.ap